In [1]:
import math
import time
import gc

import wordbatch
import numpy as np
import pandas as pd

from contextlib import contextmanager

from sklearn.metrics import roc_auc_score
from wordbatch.extractors import WordHash
from wordbatch.models import FM_FTRL

In [2]:
categorical_features = [
    'ip', 'app', 'device', 'os', 'channel',
]

numerical_features_q = [
    'PERCENT_TRUE(clicks.is_attributed)_app_1day',
    'PERCENT_TRUE(clicks.is_attributed)_device_1day',
    'PERCENT_TRUE(clicks.is_attributed)_os_1day',
    'PERCENT_TRUE(clicks.is_attributed)_channel_1day',
    'PERCENT_TRUE(clicks.is_attributed)_ip_1day',
    'PERCENT_TRUE(clicks.is_attributed)_app_device_1day',
    'PERCENT_TRUE(clicks.is_attributed)_app_os_1day',
    'PERCENT_TRUE(clicks.is_attributed)_app_channel_1day',
    'PERCENT_TRUE(clicks.is_attributed)_app_ip_1day',
    'PERCENT_TRUE(clicks.is_attributed)_device_os_1day',
    'PERCENT_TRUE(clicks.is_attributed)_device_channel_1day',
    'PERCENT_TRUE(clicks.is_attributed)_device_ip_1day',
    'PERCENT_TRUE(clicks.is_attributed)_os_channel_1day',
    'PERCENT_TRUE(clicks.is_attributed)_os_ip_1day',
]

numerical_features_l = [
#     'COUNT(clicks)_device_1day',
#     'COUNT(clicks WHERE is_attributed = True)_device_1day',
#     'COUNT(clicks)_os_1day',
#     'COUNT(clicks WHERE is_attributed = True)_os_1day',
#     'COUNT(clicks)_channel_1day',
#     'COUNT(clicks WHERE is_attributed = True)_channel_1day',
#     'COUNT(clicks)_app_1day',
#     'COUNT(clicks WHERE is_attributed = True)_app_1day',
]

numerical_features = numerical_features_q + numerical_features_l

interaction_features = [
    ['app', 'channel'],
    ['app', 'device'],
    ['app', 'os'],
    ['os', 'channel'],
    ['ip', 'app'],
#     ['device', 'PERCENT_TRUE(clicks.is_attributed)_device_1day'],
#     ['app', 'PERCENT_TRUE(clicks.is_attributed)_app_1day'],
#     ['os', 'PERCENT_TRUE(clicks.is_attributed)_os_1day'],
#     ['channel', 'PERCENT_TRUE(clicks.is_attributed)_channel_1day'],
    
#     ['device', 'COUNT(clicks)_device_1day'],
#     ['app', 'COUNT(clicks)_app_1day'],
#     ['os', 'COUNT(clicks)_os_1day'],
#     ['channel', 'COUNT(clicks)_channel_1day'],
    
#     ['device', 'COUNT(clicks WHERE is_attributed = True)_device_1day'],
#     ['app', 'COUNT(clicks WHERE is_attributed = True)_app_1day'],
#     ['os', 'COUNT(clicks WHERE is_attributed = True)_os_1day'],
#     ['channel', 'COUNT(clicks WHERE is_attributed = True)_channel_1day'],

]

In [3]:
train_filenames = [
    '../data/interim/combined/train_2017-11-07_1700_attributed2.hdf.compress',
    '../data/interim/combined/train_2017-11-07_1800_attributed2.hdf.compress',
    '../data/interim/combined/train_2017-11-07_1900_attributed2.hdf.compress',
    '../data/interim/combined/train_2017-11-07_2000_attributed2.hdf.compress',
    '../data/interim/combined/train_2017-11-07_2100_attributed2.hdf.compress',
    '../data/interim/combined/train_2017-11-07_2200_attributed2.hdf.compress',
    '../data/interim/combined/train_2017-11-07_2300_attributed2.hdf.compress',
    '../data/interim/combined/train_2017-11-08_0000_attributed2.hdf.compress',
    '../data/interim/combined/train_2017-11-08_0100_attributed2.hdf.compress',
    '../data/interim/combined/train_2017-11-08_0200_attributed2.hdf.compress',
    '../data/interim/combined/train_2017-11-08_0300_attributed2.hdf.compress',
    '../data/interim/combined/train_2017-11-08_0400_attributed2.hdf.compress',
    '../data/interim/combined/train_2017-11-08_0500_attributed2.hdf.compress',
    '../data/interim/combined/train_2017-11-08_0600_attributed2.hdf.compress',
    '../data/interim/combined/train_2017-11-08_0700_attributed2.hdf.compress',
    '../data/interim/combined/train_2017-11-08_0800_attributed2.hdf.compress',
    '../data/interim/combined/train_2017-11-08_0900_attributed2.hdf.compress',
    '../data/interim/combined/train_2017-11-08_1000_attributed2.hdf.compress',
    '../data/interim/combined/train_2017-11-08_1100_attributed2.hdf.compress',
    '../data/interim/combined/train_2017-11-08_1200_attributed2.hdf.compress',
    '../data/interim/combined/train_2017-11-08_1300_attributed2.hdf.compress',
    '../data/interim/combined/train_2017-11-08_1400_attributed2.hdf.compress',
    '../data/interim/combined/train_2017-11-08_1500_attributed2.hdf.compress',
    '../data/interim/combined/train_2017-11-08_1600_attributed2.hdf.compress'
]

validation_filenames = [
    '../data/interim/combined/train_2017-11-09_0400_attributed2.hdf.compress',
    '../data/interim/combined/train_2017-11-09_0500_attributed2.hdf.compress',
    '../data/interim/combined/train_2017-11-09_0600_attributed2.hdf.compress',
]

In [4]:
df = pd.read_hdf(train_filenames[0])

In [12]:
for c in df.columns:
    if c.startswith('PERCENT_TRUE'):
        print(c)

PERCENT_TRUE(clicks.is_attributed)_app_1day
PERCENT_TRUE(clicks.is_attributed)_device_1day
PERCENT_TRUE(clicks.is_attributed)_os_1day
PERCENT_TRUE(clicks.is_attributed)_channel_1day
PERCENT_TRUE(clicks.is_attributed)_ip_1day
PERCENT_TRUE(clicks.is_attributed)_app_device_1day
PERCENT_TRUE(clicks.is_attributed)_app_os_1day
PERCENT_TRUE(clicks.is_attributed)_app_channel_1day
PERCENT_TRUE(clicks.is_attributed)_app_ip_1day
PERCENT_TRUE(clicks.is_attributed)_device_os_1day
PERCENT_TRUE(clicks.is_attributed)_device_channel_1day
PERCENT_TRUE(clicks.is_attributed)_device_ip_1day
PERCENT_TRUE(clicks.is_attributed)_os_channel_1day
PERCENT_TRUE(clicks.is_attributed)_os_ip_1day


In [4]:
@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f'[{name}] done in {time.time() - t0:.0f} s')

In [15]:
D = 2 ** 25

wb = wordbatch.WordBatch(
    None, 
    extractor=(
        WordHash, 
        {
            "ngram_range": (1, 1), 
            "analyzer": "word",
            "lowercase": False, 
            "n_features": D,
            "norm": None, 
            "binary": True
        }),
#     minibatch_size=batchsize // 80,
#     method='threading',
    procs=24,
    freeze=True,
    timeout=1800,
    verbose=0
)

clf = FM_FTRL(
    alpha=0.05,
    beta=0.1,
    L1=0.0,
    L2=0.0,
    D=D,
    alpha_fm=0.02,
    L2_fm=0.0,
    init_fm=0.01,
    weight_fm=1.0,
    D_fm=8,
    e_noise=0.0,
    iters=3,
    inv_link="sigmoid",
    e_clip=1.0,
    threads=24,
    use_avx=1,
    verbose=0
)

In [6]:
def discretize(df):
    for c in numerical_features_q:
        df[c] = pd.qcut(df[c], 10, labels=False, duplicates='drop')
    for c in numerical_features_l:
        df[c] = pd.qcut(df[c], 100, labels=False, duplicates='drop')
    return df

In [7]:
def df2csr3(df):
    df.fillna(0)
    df = discretize(df)
    df.fillna(0)

    for i, c in enumerate(categorical_features):
        name = f"C_{i}_"
        df[name] = name + df[c].astype(str) + " "

    for i, c in enumerate(numerical_features):
        name = f"N_{i}_"
        df[name] = name + df[c].astype(str) + " "

    for i, (c1, c2) in enumerate(interaction_features):
        name = f"X_{i}_"
        df[name] = name + df[c1].astype(str) + "_" + df[c2].astype(str) + " "

    cols = [c for c in df.columns if c.startswith(('C_', 'N_', 'X_'))]
    return df[cols].sum(axis=1).values

In [8]:
def process_data(filename):
    df = pd.read_hdf(filename)
    y = df.pop('is_attributed')
    w = np.multiply([1.0 if x == 1 else 0.2 for x in y], 1)
    str_array = df2csr3(df)
    X = wb.transform(str_array)
    del df, str_array
    gc.collect()
    return X, y, w

In [13]:
%%time
X_val, y_val, _ = process_data(validation_filenames[0])

CPU times: user 7min 14s, sys: 1min 53s, total: 9min 8s
Wall time: 9min 8s


In [ ]:
i = 0
for train_filename in train_filenames:
    with timer("data"):
        X_train, y_train, w_train = process_data(train_filename)
    with timer("partial_fit"):
        clf.partial_fit(X_train, y_train, sample_weight=w_train)

    i = i + 1
    if i % 5 == 0:
        print("Test:")
        with timer("evaluate_batch test"):
            print(i, "ROC AUC:", roc_auc_score(y_val, clf.predict(X_val)))


    del X_train, y_train, w_train
    gc.collect()
    
del X_val, y_val
gc.collect()

[data] done in 154 s
[partial_fit] done in 534 s
[data] done in 89 s
[partial_fit] done in 306 s
[data] done in 60 s
[partial_fit] done in 206 s
[data] done in 50 s
[partial_fit] done in 177 s
